In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("spark_sql_basic")
sc = SparkContext(conf=conf)
sc

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/16 09:22:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local appName=spark_sql_basic>

In [2]:
movies_rdd = sc.parallelize([
    (1, ("어벤져스", "마블")),
    (2, ("슈퍼맨", "DC")),
    (3, ("배트맨", "DC")),
    (4, ("겨울왕국", "디즈니")),
    (5, ("아이언맨", "마블"))
])

attendances_rdd = sc.parallelize([
    (1, (13934592, "KR")),
    (2, (2182227,"KR")),
    (3, (4226242, "KR")),
    (4, (10303058, "KR")),
    (5, (4300365, "KR"))
])

마블 영화 중 관객수가 500만 이상인 영화 가져오기

In [5]:
# case 1 : join을 먼저 하고 Filter skwnddp

movie_att_rdd = movies_rdd.join(attendances_rdd)
movie_att_rdd.collect()
#데이터가 2중 3중으로 들어가 있어서 문제이다  각각의 데이터의 인텍스가 1개이다 (아이디), (영화 회사, 관객수 개봉 나라명) 
#그래서 1번 인덱스의 0번을 가져와야한다

# 마블만 가지고 오고 싶으면 filter를 한다
movie_att_rdd.filter(lambda x: x[1][0][1] == '마블' and x[1][1][0] >=5000000).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [7]:
# CASE 2 : Filter 먼저, Join을 나중에

filtered_movies = movies_rdd.filter(lambda x : x[1][1] == "마블")
filtered_attendances = attendances_rdd.filter(lambda x : x[1][0] >= 5000000)

filtered_movies.join(filtered_attendances).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

동일한 결과가 나오긴 하지만 filter를 먼저 수행해서 가져올 데이터를 걸러 낸 다음, join을 하는 CASE 2가 훨씬 효율적이다.

매번 이런 고민을 하기엔 시간이 너무 아깝고 개발자 마다 성능 차이도 심하다.

In [8]:
sc.stop()

# 1. SparkSession 생성하기
- `SparkContext`에 해당하며, 새로운 스파크 어플리케이션을 만들어준다.

In [29]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()
spark

In [30]:
movies = [
    (1, "어벤져스", "마블", 2012, 4, 26),
    (2, "슈퍼맨", "DC", 2013, 6, 13),
    (3, "배트맨", "DC", 2008, 8, 6),
    (4, "겨울왕국", "디즈니", 2014, 1, 16),
    (5, "아이언맨", "마블", 2008, 4, 30)
]

In [31]:
movie_schema = ["id", "name", "company", "year", "month", "day"]

2. 데이터 프레임 생성

In [32]:
movie_sdf = spark.createDataFrame(data = movies, schema = movie_schema)
movie_sdf

DataFrame[id: bigint, name: string, company: string, year: bigint, month: bigint, day: bigint]

In [33]:
# 스키마 정보 확인
movie_sdf.dtypes

[('id', 'bigint'),
 ('name', 'string'),
 ('company', 'string'),
 ('year', 'bigint'),
 ('month', 'bigint'),
 ('day', 'bigint')]

In [34]:
movie_sdf.show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  4|겨울왕국| 디즈니|2014|    1| 16|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



# 3. SparkSQL 사용하기
- 데이터 프레임을 임시 뷰로 등록을 해야 사용할 수 있다
- createOrReplaceTempView 함수를 이용해서 DataFrame에 SQL을 사용 할 수 있는 View를 만들어준다

In [40]:
movie_sdf.createOrReplaceTempView('movies')

In [41]:
# 영화 이름, 개봉 연도 가져오기
query ='''
    SELECT name, year
    FROM movies
'''

spark.sql(query).show()

+--------+----+
|    name|year|
+--------+----+
|어벤져스|2012|
|  슈퍼맨|2013|
|  배트맨|2008|
|겨울왕국|2014|
|아이언맨|2008|
+--------+----+



In [42]:
# 영화 이름, 개봉 연도 가져오기
query = '''
    SELECT name, year
    FROM movies
'''

spark.sql(query).show()


+--------+----+
|    name|year|
+--------+----+
|어벤져스|2012|
|  슈퍼맨|2013|
|  배트맨|2008|
|겨울왕국|2014|
|아이언맨|2008|
+--------+----+



In [43]:
# 2010년도 이후에 개봉한 영화의 모든 정보
query = """
    SELECT *
    FROM movies
    WHERE year > 2010
"""

spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [44]:
# 2010년도 이후에 개봉한 마블 영화의 모든 정보
query = """
    SELECT *
    FROM movies
    WHERE year > 2010
      AND company = '마블'
"""

spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
+---+--------+-------+----+-----+---+



In [45]:
# ~맨으로 끝나는 영화의 모든 정보
query = """
    SELECT *
    FROM movies
    WHERE name like "%맨"
"""

spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [46]:
# id가 3번인 영화보다 늦게 개봉한 마블영화의 모든 정보(연도만 고려)
query = """
    SELECT *
    FROM movies
    WHERE company='마블'
      AND year > ( SELECT year FROM movies WHERE id=3)
"""

spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
+---+--------+-------+----+-----+---+



# join 구현

In [47]:
attendances = [
    (1, 13934592., "KR"),
    (2, 2182227.,"KR"),
    (3, 4226242., "KR"),
    (4, 10303058., "KR"),
    (5, 4300365., "KR")
]

In [49]:
# 자료형 타입 불러오기
from pyspark.sql.types import StringType, FloatType, IntegerType

# 구조를 만들기 위한 타입 불러오기 (필수), 컬럼 순서 등
from pyspark.sql.types import StructField, StructType

In [56]:
# 리스트를 통해서 컬럼의 순서를 정리해준다// 리스트안에 StructField를 정의해준다
att_schema = StructType([
    StructField('id', IntegerType(), True), # 맨 앞에는 컬럼 이름, 타입, True(null을 허용하지 않겠다는것)
    StructField('attendance', FloatType(), True),
    StructField('country', StringType(), True)
])

In [58]:
att_df = spark.createDataFrame(data = attendances, schema = att_schema)
att_df

DataFrame[id: int, attendance: float, country: string]

In [59]:
att_df.createOrReplaceTempView('att')

In [60]:
# movies , att id를 기반으로 JOIN
query = """
SELECT *
FROM movies
JOIN att ON movies.id = att.id
ORDER BY movies.id ASC
"""

spark.sql(query).show()

+---+--------+-------+----+-----+---+---+-----------+-------+
| id|    name|company|year|month|day| id| attendance|country|
+---+--------+-------+----+-----+---+---+-----------+-------+
|  1|어벤져스|   마블|2012|    4| 26|  1|1.3934592E7|     KR|
|  2|  슈퍼맨|     DC|2013|    6| 13|  2|  2182227.0|     KR|
|  3|  배트맨|     DC|2008|    8|  6|  3|  4226242.0|     KR|
|  4|겨울왕국| 디즈니|2014|    1| 16|  4|1.0303058E7|     KR|
|  5|아이언맨|   마블|2008|    4| 30|  5|  4300365.0|     KR|
+---+--------+-------+----+-----+---+---+-----------+-------+



In [61]:
spark.stop()